In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 

import warnings 
warnings.filterwarnings("ignore") 

sns.set_style("darkgrid") 

In [2]:
dataset_path = r"C:/Users/Yashwanth G R/Desktop/internship courses/India-Tourism-Statistics-2021-Table-5.2.3.csv"

In [3]:
# Loading dataset
df = pd.read_csv(dataset_path)

df.head()

,Circle,Name of the Monument,Domestic-2019-20,Foreign-2019-20,Domestic-2020-21,Foreign-2020-21,% Growth 2021-21/2019-20-Domestic,% Growth 2021-21/2019-20-Foreign
0,Agra,Taj Mahal,4429710,645415,1259892,9034,-71.56,-98.60
1,Agra,Agra Fort,1627154,386522,371242,2810,-77.18,-99.27
2,Agra,Fatehpur Sikri,454376,184751,107835,574,-76.27,-99.69
3,Agra,Akbar Tomb Sikandra,229270,19625,99509,321,-56.60,-98.36
4,Agra,Mariam tomb Sikandra,22517,414,9765,31,-56.63,-92.51


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Circle                             178 non-null    object 
 1   Name of the Monument               178 non-null    object 
 2   Domestic-2019-20                   178 non-null    int64  
 3   Foreign-2019-20                    178 non-null    int64  
 4   Domestic-2020-21                   178 non-null    int64  
 5   Foreign-2020-21                    178 non-null    int64  
 6   % Growth 2021-21/2019-20-Domestic  177 non-null    float64
 7   % Growth 2021-21/2019-20-Foreign   176 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 11.2+ KB


In [5]:
pip install missingno

Note: you may need to restart the kernel to use updated packages.


In [7]:
missingno.matrix(df, color=(0.2, 0.6, 1), fontsize=15)
plt.show()

NameError: name 'missingno' is not defined

In [ ]:
# Getting the indexes of missing values in the column % Growth 2021-21/2019-20-Domestic 
domestic_missing_indexes = df[df['% Growth 2021-21/2019-20-Domestic'].isna()].index

# Getting the indexes of missing values in the column % Growth 2021-21/2019-20-Foreign
foreign_missing_indexes = df[df['% Growth 2021-21/2019-20-Foreign'].isna()].index

domestic_missing_indexes, foreign_missing_indexes

In [ ]:
# Filling the missing values using the equation ((df['Domestic-2020-21'] / df['Domestic-2019-2020']) - 1) * 100
df.loc[domestic_missing_indexes, '% Growth 2021-21/2019-20-Domestic'] = ((df.loc[domestic_missing_indexes, 'Domestic-2020-21'] / df.loc[domestic_missing_indexes, 'Domestic-2019-20']) - 1) * 100

# Filling the missing values using the equation ((df['Foreign-2020-21'] / df['Foreign-2019-2020']) - 1) * 100
df.loc[foreign_missing_indexes, '% Growth 2021-21/2019-20-Foreign'] = ((df.loc[foreign_missing_indexes, 'Foreign-2020-21'] / df.loc[foreign_missing_indexes, 'Domestic-2019-20']) - 1) * 100

In [ ]:
# Checking number of missing values in each feature
df.isna().sum()

In [ ]:
# Plotting countplot of "Circle" feature
plt.figure(figsize=(10,14))
sns.countplot(data=df, y="Circle", zorder=2)
plt.title("Number of city occurrences in dataset", size=19)
plt.grid(zorder=0)
plt.show()

In [ ]:
# Printing all records of Circle feature
df.Circle.values

In [ ]:
# Printing all records of "Agra" Circle with its Total
df.iloc[:10]

In [ ]:
# Calculating sum of Agra features
sum_of_Agra = df[df['Circle'] == 'Agra'][df.columns[df.dtypes != "object"]].sum()
sum_of_Agra

In [ ]:
df['Name of the Monument '] = df['Name of the Monument '].str.slice(stop=25) + '...'

In [ ]:
# Plotting countplot of "Name of the Monument " feature
plt.figure(figsize=(10,30))
sns.countplot(data=df, y="Name of the Monument ", zorder=2)
plt.title("Number of monument occurrences in dataset", size=19)
plt.grid(zorder=0)
plt.show()

In [ ]:
# Creating new dataframe without containing "Total" and "Grand Total" records
clean_df = df[~df["Circle"].isin(["Total", "Grand Total"])]

In [ ]:
# Calculating descriptive statistics of numerical features
clean_df.describe()

In [ ]:
# Plotting Boxplot of each numerical feature
for numeric_feature in df.columns[df.dtypes != "object"]:
    plt.figure(figsize=(15,6))
    sns.boxplot(data=clean_df, x=numeric_feature);
    plt.title(f"Boxplot of {numeric_feature} feature", size=19)
    plt.xscale('log')
    plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encoding discrete features
numerical_df = clean_df.copy()
numerical_df["Circle"] = LabelEncoder().fit_transform(clean_df["Circle"])
numerical_df["Name of the Monument "] = LabelEncoder().fit_transform(clean_df["Name of the Monument "])

# Displaying first 5 rows
numerical_df.head()

In [ ]:
from scipy.stats import pearsonr
import matplotlib.cm as cm

def corrfunc(x, y, ax=None, **kws):
    """Function to plot the correlation coefficient of a plot"""
    r, _ = pearsonr(x, y)
    cmap = sns.diverging_palette(-1, 1, as_cmap=True)
    cmap = cm.get_cmap('coolwarm')
    ax = ax or plt.gca()
    ax.set_facecolor(color=cmap(r))
    ax.annotate(f'{r:.2f}', xy=(.14, .4) if r>0 else (.05, .4), xycoords=ax.transAxes,
                fontsize=50, alpha=0.7, color="w")
    
# Plotting pairplot
g = sns.pairplot(numerical_df)
g.map_lower(corrfunc)
g.map_lower(sns.scatterplot, color='black')
plt.show()

In [ ]:
# Creating new column "total-2019-20-visitors" and "total-2020-21-visitors"
clean_df['total-2019-20-visitors'] = clean_df[['Domestic-2019-20', 'Foreign-2019-20']].sum(axis=1)
clean_df['total-2020-21-visitors'] = clean_df[['Domestic-2020-21', 'Foreign-2020-21']].sum(axis=1)

# Sorting data by 'total-2019-20-visitors'
clean_df = clean_df.sort_values(by='total-2019-20-visitors', ascending=False)

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(30,70))

# Plotting barplot of 2019-20 Visitors
plt.subplot(1,2,1)
sns.barplot(y='Name of the Monument ', x="total-2019-20-visitors",data=clean_df, color='purple', label='Foreign')
sns.barplot(y='Name of the Monument ', x='Domestic-2019-20', data=clean_df, color='orange', label='Domestic')
plt.title("2019-20 Visitors", fontsize=40)
plt.legend(bbox_to_anchor=(1, 0.99), loc='upper right', prop={'size': 30});
plt.xticks(fontsize=30)
plt.xlabel('Number of Visitors', fontsize=40)
plt.yticks(fontsize=30)
plt.ylabel('Name of the Monument', fontsize=40)

# Plotting barplot of 2020-21 Visitors
plt.subplot(1,2,2)
sns.barplot(y='Name of the Monument ', x="total-2020-21-visitors", data=clean_df, color='purple', label='Foreign')
sns.barplot(y='Name of the Monument ', x='Domestic-2020-21', data=clean_df, color='orange', label='Domestic')
plt.title("2020-21 Visitors", fontsize=40)
plt.legend(bbox_to_anchor=(1, 0.99), loc='upper right', prop={'size': 30});
plt.xticks(fontsize=30)
plt.xlabel('Number of Visitors', fontsize=40)
plt.ylabel('Name of the Monument', fontsize=40)

fig.suptitle("Number of Tourists Based on Monument in Given Years", fontsize=40, fontweight='bold');

In [ ]:
# Plotting barplots of total visitors during those years
plt.figure(figsize=(30,70))
ax = sns.barplot(y='Name of the Monument ', x="total-2019-20-visitors",data=clean_df, color='#00BFFF', label='2019-20 Visitors', alpha=0.7)
sns.barplot(y='Name of the Monument ', x="total-2020-21-visitors", data=clean_df, color='red', label='2020-21 Visitors', alpha=0.5)
plt.title("Diffrence of Number of Tourists Based on Monument in Given Years", fontsize=40, fontweight='bold');
plt.legend(bbox_to_anchor=(1, 0.81), loc='upper right', prop={'size': 30});
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel('Number of Visitors', fontsize=40)
plt.ylabel('Name of the Monument', fontsize=40);
plt.xscale("log")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = clean_df[['total-2019-20-visitors']]
y = clean_df['total-2020-21-visitors']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create a Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the model coefficients and evaluation metrics
print("Linear Regression Coefficients:", model.coef_)
print("Mean Squared Error:", mse)
print("R-squared:", r2)